In [3]:
# default_exp core

# to_timemap

> Exports a timemap-ready XLSX from some other data.

In [91]:
#export
import json
import xlsxwriter
from datetime import datetime
from fastscript import *

In [100]:
#export
DEFAULT_HEADERS = [
    "id",
    "description",
    "location",
    "date",
    "time",
    "latitude",
    "longitude",
    "category",
    "filter0",
    "colour"
]

In [113]:
import random

def distinct_colors(n):
    ret = []
    r = int(random.random() * 256)
    g = int(random.random() * 256)
    b = int(random.random() * 256)
    step = 256 / n
    for i in range(n):
        r += step
        g += step
        b += step
        r = int(r) % 256
        g = int(g) % 256
        b = int(b) % 256
        ret.append(f"rgb({r},{g},{b}") 
    return ret

In [114]:
#export
def dt(d: str) -> datetime:
    return datetime.strptime(d, '%Y-%m-%dT%H:%M:%SZ')

def getDate(item: str) -> datetime:
    lspt = item.get('lastSpotted')
    if lspt is not None:
        return dt(lspt)
    startTime = item.get('startTime')
    if startTime is not None:
        return dt(startTime)
    endTime = item.get('endTime')
    if endTime is not None:
        return dt(endTime)
    raise Exception("JSON value doesn't have lastSpotted, startTime, or endTime")
    
def from_fleming(jdata, color):
    headers = DEFAULT_HEADERS
    odata = []
    past_locations = {}
    for idx, d in enumerate(jdata):
        thedate = getDate(d)
        lat = d['coordinates']['lat']
        lon = d['coordinates']['lon']
        locname = f"location{idx}"
        k = f"{lat}_{lon}"
        if past_locations.get(k) is not None:
            locname = past_locations[k]
        else:
            past_locations[f"{lat}_{lon}"] = locname
        rdata = [
            idx,
            d['targetId'],
            locname,
            datetime.strftime(thedate, '%m/%d/%Y'),
            datetime.strftime(thedate, '%H:%M:%S'),
            lat,
            lon,
            "alpha", # just one category
            d['targetId'],
            color
            
        ]
        odata.append(rdata)

    return headers, odata

In [115]:
#export
CONVERTER = from_fleming

def write_sheet(sheet, hdrs, rows):
    row = 0
    col = 0

    for hdr in hdrs:
        sheet.write(row, col, hdr)
        col += 1

    row = 1
    col = 0

    for rdata in rows:
        col = 0
        for cell in rdata:
            sheet.write(row, col, cell)
            col += 1
        row += 1
    
@call_parse
def write_xlsx_from_json(inp:Param("Input file", str),
                 outp:Param("Output file", str)="out.xlsx",
                 color:Param("The color of events", str)="red",
                 tabname:Param('Name of tab', str)="export_events"):
    workbook = xlsxwriter.Workbook(outp)
    worksheet = workbook.add_worksheet(tabname)

    with open(inp, 'r') as f:
        data = json.load(f)

    conv_data = CONVERTER(data, color)
    headers = conv_data[0]
    row_data = conv_data[1]

    write_sheet(worksheet, headers, row_data)
    
    workbook.close()

## Test
Running on sample.json from notebook.

In [116]:
import os
from pathlib import Path
samplef = Path(os.getcwd()).parents[0] / "sample.json"
# main(samplef)

# Generate .xlsx files for all .jsons
Store the names of files processed in `created_sheets` for the next step.

In [117]:
desktop = list(Path(os.getcwd()).parents)[-3] / 'Desktop'
targets = desktop/'TargetsIds'
countries = [Path(x) for x in targets.glob("**/*") if x.is_dir()]
output = desktop/'TargetsDatasheets'
output_xlsx = output/'datasheet-server'/'data'
output_xlsx.mkdir(parents=True, exist_ok=True)
created_sheets = []

In [118]:
bhr_folder = countries[0]
bhr_files = [x for x in bhr_folder.glob("**/*") if x.is_file()]
colors = distinct_colors(len(bhr_files))
print(colors)
for idx, f in enumerate(bhr_files):
    # create a f"{f.name}.xlsx" in output using main()
    write_xlsx_from_json(f, output_xlsx/f"bhr_{f.stem}.xlsx", colors[idx])
    created_sheets.append(f.stem)

# make a metadata sheet for categories and filters
workbook = xlsxwriter.Workbook(output_xlsx/"metadata.xlsx")
write_sheet(workbook.add_worksheet('export_categories'),
            ['category', 'description'],
            [['alpha', 'the only category...']])
write_sheet(workbook.add_worksheet('export_filters'),
            ['None'],
            [[x] for x in created_sheets])
workbook.close()

['rgb(152,5,197', 'rgb(203,56,248', 'rgb(254,107,43', 'rgb(49,158,94', 'rgb(100,209,145']


# Generate timemap and datasheet-server templates

In [106]:
import pandas as pd
def derive_latlon(sheets):
    # for the time being, just pick first
    sheet =  pd.read_excel(output_xlsx / f"bhr_{sheets[0]}.xlsx", sheet_name='export_events')
    for idx, row in sheet[['latitude', 'longitude']].iterrows():
        return f"[{row.latitude}, {row.longitude}]"

In [107]:
from string import Template

# event_exts, category_ext, map_anchor
tm_template = Template("""module.exports = {
  title: 'example',
  display_title: 'Target Trace',
  SERVER_ROOT: 'http://localhost:4040',
  EVENTS_EXT: [
      $event_exts
  ],
  CATEGORIES_EXT: '$categories_ext',
  SOURCES_EXT: '',
  NARRATIVES_EXT: '',
  FILTERS_EXT: '$filters_ext',
  SITES_EXT: '',
  DATE_FMT: 'MM/DD/YYYY',
  TIME_FMT: 'hh:mm:ss',

  store: {
    app: {
      map: {
        anchor: $map_anchor,
        maxZoom: 15,
        minZoom: 3,
        startZoom: 10
      },
      timeline: {
        range: [
          new Date('2020-03-01T00:00:00Z'),
          new Date('2020-04-01T00:00:00.000Z')
        ],
        rangeLimits: [
          new Date('2019-01-01T00:00:00.000Z'),
          new Date('2020-07-01T00:00:00.000Z')
        ],
        dimensions: {}
      }
    },
    features: {
      USE_COVER: false,
      USE_CATEGORIES: true,
      USE_FILTERS: true,
      USE_SITES: false,
      USE_SOURCES: false,
      USE_NARRATIVES: false,
      USE_SEARCH: false
    }
  }
}
""")

single_xlsx = Template("""{ name: '$name', path: 'data/bhr_$name.xlsx', tabs: timemap.default }""")

dss_template = Template("""import { timemap } from './lib'

export default {
  gsheets: [],
  xlsx: [
      { name: 'metadata', path: 'data/metadata.xlsx', tabs: timemap.default },
      $xlsx_sheets
  ]
}
""")

dss_config = dss_template.substitute(
    xlsx_sheets=',\n'.join([single_xlsx.substitute(name=s) for s in created_sheets])
)

tm_config = tm_template.substitute(
    event_exts=", ".join([f"'/api/{x}/export_events/deeprows'" for x in created_sheets]),
    categories_ext="/api/metadata/export_categories/rows",
    filters_ext="/api/metadata/export_filters/tree",
    map_anchor=derive_latlon(created_sheets)
)

with (output/'timemap'/'config.js').open('w', encoding='utf-8') as f:
    f.write(tm_config)
    
with (output/'datasheet-server'/'src'/'config.js').open('w', encoding='utf-8') as f:
    f.write(dss_config)

In [81]:
#hide
from nbdev.showdoc import *

In [74]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
